In [1]:
import csv
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader, Dataset

In [2]:
hotel_reviews = pd.read_csv(r"C:\Users\Damja\OneDrive\Damjan\HS22\NLP UZH\Exercise2\tripadvisor_hotel_reviews.csv")
#scifi = pd.read_csv(r"C:\Users\Damja\OneDrive\Damjan\HS22\NLP UZH\Exercise2\scifi.txt")

# Exercise 1

Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning. It is a model that tries to predict words given the context of a few words before and a few words after the target word. This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic. Typcially, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model. Usually, this is referred to as pretraining embeddings. It almost always helps performance a couple of percent.

The CBOW model is as follows. Given a target word wi and an N context window on each side, wi−1,…,wi−N and wi+1,…,wi+N, referring to all context words collectively as C, CBOW tries to minimize

−logp(wi|C)=−logSoftmax(A(∑w∈Cqw)+b)

where qw is the embedding for word w.

Implement this model in Pytorch by filling in the class below. Some tips:

    Think about which parameters you need to define.
    Make sure you know what shape each operation expects. Use .view() if you need to reshape.


In [15]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = dict((v, k) for k, v in word_to_ix.items())
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))


# create your model and train.  here are some functions to help you make
# the data ready for use by your module

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

tensor([18, 20, 30, 15])

In [19]:
print(ix_to_word)

print(word_to_ix)

{0: 'In', 1: 'idea', 2: 'programs', 3: 'People', 4: 'the', 5: 'a', 6: 'Computational', 7: 'of', 8: 'beings', 9: 'they', 10: 'process.', 11: 'program.', 12: 'As', 13: 'evolve,', 14: 'effect,', 15: 'study', 16: 'directed', 17: 'pattern', 18: 'We', 19: 'computer', 20: 'are', 21: 'processes.', 22: 'data.', 23: 'we', 24: 'The', 25: 'spells.', 26: 'manipulate', 27: 'computers.', 28: 'our', 29: 'by', 30: 'to', 31: 'processes', 32: 'is', 33: 'abstract', 34: 'about', 35: 'evolution', 36: 'process', 37: 'direct', 38: 'computational', 39: 'that', 40: 'called', 41: 'conjure', 42: 'spirits', 43: 'other', 44: 'inhabit', 45: 'rules', 46: 'create', 47: 'things', 48: 'with'}
{'In': 0, 'idea': 1, 'programs': 2, 'People': 3, 'the': 4, 'a': 5, 'Computational': 6, 'of': 7, 'beings': 8, 'they': 9, 'process.': 10, 'program.': 11, 'As': 12, 'evolve,': 13, 'effect,': 14, 'study': 15, 'directed': 16, 'pattern': 17, 'We': 18, 'computer': 19, 'are': 20, 'processes.': 21, 'data.': 22, 'we': 23, 'The': 24, 'spells.

In [5]:
# create Dataset class for transforming out dataset
# takes the loaded data and transforms it into a Dataset object to pass to the dataloader
# we could also include the import here but we will not

class CBOW_Dataset(Dataset):
    
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = make_context_vector(data[idx][0], word_to_ix)
        y = torch.tensor(word_to_ix[data[idx][1]], dtype=torch.long)
        return x, y

In [6]:
## THIS ONE DOESNT WORK GOD KNOWS WHY SUCH MINOR DIFFERENCES ONLY F THIS
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_size):
        
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                     embedding_dim=embedding_size)
        self.l1 = nn.Linear(in_features=embedding_size,
                            out_features=vocab_size)

    def forward(self, inputs):
        embeddings = torch.sum(self.embedding(inputs), dim=1) # [200, 4, 50] => [200, 50]
        # embeds = self.embeddings(inputs).view((batch_size, -1))
        y_out = self.l1(embeddings) 
        log_probs = F.log_softmax(y_out, dim=1)
        return log_probs

In [7]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_size):
        
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                     embedding_dim=embedding_size)
        self.fc1 = nn.Linear(in_features=embedding_size,
                            out_features=vocab_size)

    def forward(self, inputs):
        x_embedded = sum(self.embedding(inputs))
        y_out = self.fc1(x_embedded)
        m = nn.LogSoftmax()
        y_out = m(y_out).reshape(-1)
        return y_out


### Let's try and train this model

In [10]:
data_loader = CBOW_Dataset(data)

In [11]:
import torch.optim as optim
model = CBOW(vocab_size, embedding_size=30)
loss_func = torch.nn.NLLLoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

num_epochs = 10

In [12]:
# train loop
num_batches = len(data_loader)

for epoch in range(1, num_epochs+1):
    for i, d in enumerate(data_loader):
        optimizer.zero_grad()
        y_pred = model(d[0])        
        loss = loss_func(y_pred, d[1])
        loss_batch = loss.item()
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch}/{num_epochs}], batch: [{i}/{num_batches}, loss: {loss_batch:.4f}]')

C:\Users\Damja\AppData\Local\Temp\ipykernel_22776\1816555312.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  y_out = m(y_out).reshape(-1)


Epoch [1/10], batch: [0/58, loss: 3.2026]
Epoch [1/10], batch: [1/58, loss: 6.7962]
Epoch [1/10], batch: [2/58, loss: 3.0113]
Epoch [1/10], batch: [3/58, loss: 4.4938]
Epoch [1/10], batch: [4/58, loss: 2.2399]
Epoch [1/10], batch: [5/58, loss: 6.2244]
Epoch [1/10], batch: [6/58, loss: 4.1509]
Epoch [1/10], batch: [7/58, loss: 5.1393]
Epoch [1/10], batch: [8/58, loss: 4.0848]
Epoch [1/10], batch: [9/58, loss: 7.1552]
Epoch [1/10], batch: [10/58, loss: 5.8895]
Epoch [1/10], batch: [11/58, loss: 4.2735]
Epoch [1/10], batch: [12/58, loss: 3.9668]
Epoch [1/10], batch: [13/58, loss: 6.1331]
Epoch [1/10], batch: [14/58, loss: 7.8882]
Epoch [1/10], batch: [15/58, loss: 2.9878]
Epoch [1/10], batch: [16/58, loss: 3.4766]
Epoch [1/10], batch: [17/58, loss: 6.0062]
Epoch [1/10], batch: [18/58, loss: 3.1658]
Epoch [1/10], batch: [19/58, loss: 4.3190]
Epoch [1/10], batch: [20/58, loss: 4.7163]
Epoch [1/10], batch: [21/58, loss: 2.8541]
Epoch [1/10], batch: [22/58, loss: 2.6672]
Epoch [1/10], batch: 

Epoch [8/10], batch: [38/58, loss: 3.2309]
Epoch [8/10], batch: [39/58, loss: 2.9018]
Epoch [8/10], batch: [40/58, loss: 2.9649]
Epoch [8/10], batch: [41/58, loss: 1.7677]
Epoch [8/10], batch: [42/58, loss: 1.3538]
Epoch [8/10], batch: [43/58, loss: 2.1766]
Epoch [8/10], batch: [44/58, loss: 1.3906]
Epoch [8/10], batch: [45/58, loss: 3.5733]
Epoch [8/10], batch: [46/58, loss: 2.8215]
Epoch [8/10], batch: [47/58, loss: 2.9303]
Epoch [8/10], batch: [48/58, loss: 2.7174]
Epoch [8/10], batch: [49/58, loss: 1.2820]
Epoch [8/10], batch: [50/58, loss: 2.4885]
Epoch [8/10], batch: [51/58, loss: 3.8988]
Epoch [8/10], batch: [52/58, loss: 1.7122]
Epoch [8/10], batch: [53/58, loss: 2.3607]
Epoch [8/10], batch: [54/58, loss: 1.3390]
Epoch [8/10], batch: [55/58, loss: 1.9818]
Epoch [8/10], batch: [56/58, loss: 1.6602]
Epoch [8/10], batch: [57/58, loss: 2.2851]
Epoch [9/10], batch: [0/58, loss: 1.2482]
Epoch [9/10], batch: [1/58, loss: 4.4485]
Epoch [9/10], batch: [2/58, loss: 1.3257]
Epoch [9/10], 

In [13]:
y_pred

tensor([-5.3167, -2.3323, -4.8522, -3.3945, -4.0869, -5.6906, -5.8298, -2.4287,
        -4.3467, -6.1285, -5.7904, -5.9563, -5.7445, -4.5328, -6.3826, -4.0178,
        -4.9200, -3.7772, -6.0492, -2.1705, -3.9264, -4.7247, -4.2075, -3.5732,
        -5.9498, -5.4486, -4.6216, -4.4562, -4.8541, -5.8288, -4.1564, -6.3319,
        -5.1742, -5.9197, -3.7224, -4.8586, -5.2240, -5.6489, -4.8130, -4.9374,
        -4.6739, -2.9141, -2.7901, -4.4224, -4.9281, -3.8123, -4.7366, -6.4402,
        -1.7427], grad_fn=<ReshapeAliasBackward0>)

In [31]:
context = ['We', 'are', 'to', 'study']
context_vector = make_context_vector(context, word_to_ix)
pred = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(pred).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['We', 'are', 'to', 'study']

Prediction: about


C:\Users\Damja\AppData\Local\Temp\ipykernel_22776\1816555312.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  y_out = m(y_out).reshape(-1)
